In [213]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [197]:
def removenote(cities, tag):
    templist = []
    for a in cities[tag]:
        templist.append(a.split('[')[0])
    cities[tag] = templist
    return cities

# Now fix the team names in cities df
def findorgname(team):
    if team == '':
        return '-'
    for x in all_nhl_teams:
        if team in x:
            return x
    return '-'

In [198]:
nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = removenote(cities,'NHL')
cities = cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
cities = cities.drop(['NFL','MLB','NBA'],axis=1)


# replace new york city teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'New York City']['NHL'][0])
cities.iloc[0]['NHL'] = x

# replace LA teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'Los Angeles']['NHL'][1])
cities.iloc[1]['NHL'] = x

cities = cities.explode('NHL')
cities

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,Rangers
0,New York City,20153634,Islanders
0,New York City,20153634,Devils
1,Los Angeles,13310447,Kings
1,Los Angeles,13310447,Ducks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars
5,"Washington, D.C.",6131977,Capitals
6,Philadelphia,6070500,Flyers


In [199]:
nhl_df = nhl_df.drop(0)
nhl_df['team'] = nhl_df['team'].apply(lambda x: x.replace('*',''))
all_nhl_teams = set(nhl_df['team'].to_list())
temp_nhl_list = []
for team in cities['NHL']:
    actual_name = findorgname(team)
    if actual_name == 'Phoenix Coyotes':
        actual_name = 'Arizona Coyotes'
    temp_nhl_list.append(actual_name)
    
cities['NHL'] = temp_nhl_list

In [200]:
nhl_df.loc[nhl_df[nhl_df['team'] == 'Phoenix Coyotes'].index,'team'] = 'Arizona Coyotes'


In [201]:
nhl_df = nhl_df.set_index('team')

In [202]:
cities = cities.rename({'NHL':'team'}, axis = 1)
cities = cities.set_index('team')
cities

,Metropolitan area,Population 2016
team,,
New York Rangers,New York City,20153634
New York Islanders,New York City,20153634
New Jersey Devils,New York City,20153634
Los Angeles Kings,Los Angeles,13310447
Anaheim Ducks,Los Angeles,13310447
San Jose Sharks,San Francisco Bay Area,6657982
Chicago Blackhawks,Chicago,9512999
Dallas Stars,Dallas–Fort Worth,7233323
Washington Capitals,"Washington, D.C.",6131977


In [203]:
merged = pd.merge(cities, nhl_df, how='inner',left_index = True, right_index=True)

merged = merged[merged['year'] == 2018]
area_indexed = merged.reset_index()

area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)
area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})



In [204]:
# cities[:]

In [205]:
# merged[merged.index=='Phoenix Coyotes']

In [206]:
area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
area_indexed[:]

,team,Metropolitan area,Population 2016,W,L,win_loss_ratio
0,Anaheim Ducks,Los Angeles,13310447,44,25,0.637681
1,Arizona Coyotes,Phoenix,4661537,29,41,0.414286
2,Boston Bruins,Boston,4794447,50,20,0.714286
3,Buffalo Sabres,Buffalo,1132804,25,45,0.357143
4,Calgary Flames,Calgary,1392609,37,35,0.513889
5,Carolina Hurricanes,Raleigh,1302946,36,35,0.507042
6,Chicago Blackhawks,Chicago,9512999,33,39,0.458333
7,Colorado Avalanche,Denver,2853077,43,30,0.589041
8,Columbus Blue Jackets,Columbus,2041520,45,30,0.600000
9,Dallas Stars,Dallas–Fort Worth,7233323,42,32,0.567568


In [220]:
# group by city, sum up the win loss
a = 0
win_loss_by_region = []
population_by_region = []
for group, frame in area_indexed.groupby('Metropolitan area'):
    win = np.sum(frame['W'])
    loss = np.sum(frame['L'])
    win_loss_by_region.append(win/(loss+win))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)

print('len',len(population_by_region))
print(stats.pearsonr(population_by_region, win_loss_by_region))

len 28
(0.012308996455744264, 0.9504308637909502)


In [217]:
# nhl_df[nhl_df.index=='Arizona Coyotes']

In [210]:
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.012308996455744264

In [222]:
# group by city, sum up the win loss
a = 0
win_loss_by_region = []
population_by_region = []
for group, frame in area_indexed.groupby('Metropolitan area'):
#     print(np.mean(frame['win_loss_ratio']))
    win = np.sum(frame['W'])
    loss = np.sum(frame['L'])
    win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)

print('len',len(population_by_region))
print(stats.pearsonr(population_by_region, win_loss_by_region))

0.7142857142857143
0.35714285714285715
0.5138888888888888
0.4583333333333333
0.6
0.5675675675675675
0.589041095890411
0.43478260869565216
0.47368421052631576
0.68
0.622894633764199
0.5945945945945946
0.6338028169014085
0.42028985507246375
0.7464788732394366
0.5182014205986808
0.39436619718309857
0.6176470588235294
0.4142857142857143
0.618421052631579
0.5070422535211268
0.625
0.5789473684210527
0.7012987012987013
0.6533333333333333
0.43661971830985913
0.6533333333333333
0.7222222222222222
len 28
(0.012308996455744264, 0.9504308637909502)
